# Importing Libraries & Reading JSON

In [1]:
import sys
import os
import json
import itertools

json_list = json.load(open('parameters_json_file.json', 'r'))
print("parameters_json_file.json just opened")

data = json_list["data"]
parameters = json_list["parameters"]
fixed_parameters = parameters["fixed"]
iterable_parameters = parameters["iterable"]
print(iterable_parameters)

parameters_json_file.json just opened
{'rnn_dim': [100, 150, 200, 300], 'char_rnn': [25, 50], 'word_min_freq': [0, 5, 10], 'train_batch_size': [4, 8, 16]}


## Parameters Combinations

In [2]:
listofvalues=iterable_parameters.values()
args_list = []
for it_key in iterable_parameters:
    print(it_key, "\t", iterable_parameters[it_key])
    args_list.append(it_key)
parameter_combinations = list(itertools.product(*listofvalues))
print("\nNumber of combinations to try:", len(parameter_combinations))


rnn_dim 	 [100, 150, 200, 300]
char_rnn 	 [25, 50]
word_min_freq 	 [0, 5, 10]
train_batch_size 	 [4, 8, 16]

Number of combinations to try: 72


## Class: Command - add_arg()

In [3]:
class Command_string:
    def __init__(self, initial_string = "python"):
        self.string = initial_string
    
    def change(self, new_string):
        self.string = new_string
    
    def add_arg_str(self, string):
        self.string += " " + string
        
    def add_arg_dic(self, key, dic):
        self.string += " --" + key + " " + str(dic[key])
    
    def add_arg_val(self, key, val):
        self.string += " --" + key + " " + str(val)

## Commands String

### Fixed string part

In [4]:
fixed_command = Command_string()
fixed_command.add_arg_str(json_list["train.py"])
for path_key in data:
    fixed_command.add_arg_dic(path_key, data)
for fixed_par_key in fixed_parameters:
    fixed_command.add_arg_dic(fixed_par_key, fixed_parameters)
print(fixed_command.string)

python /disks/md0-4T/users/b.mgm/model/train.py --data_path /disks/md0-4T/users/b.mgm/data/CRF --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path /disks/md0-4T/users/b.mgm/data/embedding/pretrained_embedding.txt --char_emb_dim 25 --word_emb_dim 100 --val_batch_size 64 --max_epochs 100


### Changing part (due to arguments combinations) --> list of strings

In [5]:
commands_list = []
tmp_command = Command_string()
#print(tmp_command.string)

for comb in parameter_combinations:
    tmp_command.change(fixed_command.string)
    comb_num = parameter_combinations.index(comb)
    name = "training_" + str(comb_num) + "_"
    for it_par_key in iterable_parameters:
        key_index = list(iterable_parameters.keys()).index(it_par_key)
        tmp_command.add_arg_val(it_par_key, comb[key_index])
        #print(key_index, it_par_key, comb[key_index])
        name += "__" + it_par_key + "_" + str(comb[key_index])
        checkpoint_dir = json_list["checkpoints"] + "/" + name
        tmp_command.add_arg_val("checkpoint", checkpoint_dir)
        logs_dir = json_list["logs"]
    #print("--> \n", tmp_command.string, "\n")
    #print(name)
    post_command = " | tee -a " + logs_dir + "/" + name + ".log"
    commands_list.append(tmp_command.string + post_command)
print("List length:", len(commands_list))
index_to_show = 1
print("List command %d (%d/%d):\n%s" % (index_to_show, index_to_show+1, len(commands_list), commands_list[index_to_show]))

List length: 72
List command 1 (2/72):
python /disks/md0-4T/users/b.mgm/model/train.py --data_path /disks/md0-4T/users/b.mgm/data/CRF --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path /disks/md0-4T/users/b.mgm/data/embedding/pretrained_embedding.txt --char_emb_dim 25 --word_emb_dim 100 --val_batch_size 64 --max_epochs 100 --rnn_dim 100 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_1___rnn_dim_100 --char_rnn 25 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_1___rnn_dim_100__char_rnn_25 --word_min_freq 0 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_1___rnn_dim_100__char_rnn_25__word_min_freq_0 --train_batch_size 8 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_1___rnn_dim_100__char_rnn_25__word_min_freq_0__train_batch_size_8 | tee -a /disks/md0-4T/users/b.mgm/logs/training_1___rnn_dim_100__char_rnn_25__word_min_freq_0__train_batch_size_8.log


# Output File

## Opening File

In [6]:
print("Opening writing file")
out_file_name = json_list["output_name"]
print("Name of file:", out_file_name)
out_file = open(out_file_name, 'w') # Passar a 'x' per a que creï un nou fitxer i sinó ERROR

Opening writing file
Name of file: training_commands.lst


## Printing String into File

In [7]:
for line in commands_list:
    out_file.write(line + "\n")
out_file.close()

In [8]:
out_file = open(out_file_name, 'r')
print(out_file.read())
out_file.close()

python /disks/md0-4T/users/b.mgm/model/train.py --data_path /disks/md0-4T/users/b.mgm/data/CRF --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path /disks/md0-4T/users/b.mgm/data/embedding/pretrained_embedding.txt --char_emb_dim 25 --word_emb_dim 100 --val_batch_size 64 --max_epochs 100 --rnn_dim 100 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_0___rnn_dim_100 --char_rnn 25 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_0___rnn_dim_100__char_rnn_25 --word_min_freq 0 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_0___rnn_dim_100__char_rnn_25__word_min_freq_0 --train_batch_size 4 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_0___rnn_dim_100__char_rnn_25__word_min_freq_0__train_batch_size_4 | tee -a /disks/md0-4T/users/b.mgm/logs/training_0___rnn_dim_100__char_rnn_25__word_min_freq_0__train_batch_size_4.log
python /disks/md0-4T/users/b.mgm/model/train.py --data_path /disks/md0-4T/users/b.m